In [10]:
import pickle 
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

In [11]:
with open('preprocessed_data/preprocessed_general.pkl', 'rb') as f:
    chosen_stocks = pickle.load(f)

We now have our stock data with features for each stock. The next step for us is to split the data into train and test/validation to fit a scaler only on the training set.
We do that so that the scaler does not get any information about the range the values may lie in in the test or validation set.

In [12]:
end_train_set = chosen_stocks[0][0].index[int((chosen_stocks[0][0].shape[0] - 1) * 0.8)]
end_validation_set = chosen_stocks[0][0].index[int((chosen_stocks[0][0].shape[0] - 1) * 0.9)]
print(end_train_set)
print(end_validation_set)

2024-05-13 00:00:00
2024-11-05 00:00:00


In [13]:
chosen_stocks[2][9]

,close,high,low,trade_count,open,volume,vwap,symbol,rsi,roc_9,...,personal_consumption,industrial_production,federal_funds_rate,10Y_treasury_rate,retail_sales,housing_starts,usd_eur,usd_gbp,usd_jpy,target
2020-06-19,237.82,242.9800,235.4949,26344,242.110,2629398,239.190730,ADSK,69.120755,1.845745,...,14208.163636,93.793082,0.086364,0.70,523298.181818,1431.454545,1.1189,1.2362,106.89,240.88
2020-06-22,240.88,241.1300,236.0000,18318,236.710,1423163,239.653226,ADSK,70.863458,2.874226,...,14220.118182,93.944409,0.086818,0.71,523673.909091,1443.272727,1.1260,1.2447,106.84,241.62
2020-06-23,241.62,247.1400,241.4550,21197,242.910,1630635,243.784691,ADSK,71.285497,2.126041,...,14232.072727,94.095736,0.087273,0.72,524049.636364,1455.090909,1.1322,1.2531,106.44,233.44
2020-06-24,233.44,242.6747,231.4382,32961,240.290,2035221,234.328567,ADSK,60.801324,6.544957,...,14244.027273,94.247064,0.087727,0.69,524425.363636,1466.909091,1.1272,1.2432,106.83,234.47
2020-06-25,234.47,234.8100,228.3700,26754,233.500,2079323,233.360467,ADSK,61.567797,5.162361,...,14255.981818,94.398391,0.088182,0.68,524801.090909,1478.727273,1.1221,1.2406,107.21,232.74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-04-30,274.25,274.8700,267.6100,30330,270.405,1226321,272.957322,ADSK,58.404123,5.415898,...,20669.500000,103.605109,4.330000,4.17,715715.454545,1262.181818,1.1349,1.3344,142.63,273.49
2025-05-01,273.49,278.5700,273.1350,34752,276.140,1303907,275.150627,ADSK,57.603508,5.403322,...,20669.500000,103.594800,4.330000,4.25,715417.000000,1256.000000,1.1279,1.3273,145.48,279.99
2025-05-02,279.99,281.1900,276.3200,29402,278.300,959634,279.598063,ADSK,62.356381,10.115232,...,20669.500000,103.594800,4.330000,4.33,715417.000000,1256.000000,1.1330,1.3283,144.52,280.07
2025-05-05,280.07,283.2700,278.1900,22970,278.190,894591,280.559063,ADSK,62.412232,7.248985,...,20669.500000,103.594800,4.330000,4.36,715417.000000,1256.000000,1.1315,1.3284,143.98,278.64


*** WARNING *** This Pipeline somehow confuses the column symbol with usd_jpy. Still to fix.

In [14]:
# Create lists to hold the data all bins
preprocessed_NN_data = []
for i in range(len(chosen_stocks)):
    preprocessed_NN_bin = []
    for j in range(len(chosen_stocks[i])):
        #Split into train, validation and test sets (data is already sorted by date)
        X_train = chosen_stocks[i][j].loc[:end_train_set, chosen_stocks[i][j].columns != 'target']
        y_train = chosen_stocks[i][j].loc[:end_train_set, 'target']
        X_validation = chosen_stocks[i][j].loc[end_train_set:end_validation_set, chosen_stocks[i][j].columns != 'target']
        y_validation = chosen_stocks[i][j].loc[end_train_set:end_validation_set, 'target']
        X_test = chosen_stocks[i][j].loc[end_validation_set:, chosen_stocks[i][j].columns != 'target']
        y_test = chosen_stocks[i][j].loc[end_validation_set:, 'target']

        # Scale Features based on train set
        # We drop the 'symbol' column before scaling and add it back after scaling
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train.drop(columns=['symbol'], inplace=False))
        X_validation_scaled = scaler.transform(X_validation.drop(columns=['symbol'], inplace=False))
        X_test_scaled = scaler.transform(X_test.drop(columns=['symbol'], inplace=False))
        X_train_scaled = np.concatenate((X_train_scaled, X_train[['symbol']].values), axis=1)
        X_validation_scaled = np.concatenate((X_validation_scaled, X_validation[['symbol']].values), axis=1)
        X_test_scaled = np.concatenate((X_test_scaled, X_test[['symbol']].values), axis=1)

        # Put back into dfs so we have the original indices and columns for windowing
        X_train_scaled_df = pd.DataFrame(X_train_scaled, index=X_train.index, columns=X_train.columns)
        X_validation_scaled_df = pd.DataFrame(X_validation_scaled, index=X_validation.index, columns=X_validation.columns) 
        X_test_scaled_df = pd.DataFrame(X_test_scaled, index=X_test.index, columns=X_test.columns)


        # We concatenate the dataframes to create a single dataset for windowing so that we dont lose window.size() windows at the start of each set but only once.
        X = pd.concat([X_train_scaled_df, X_validation_scaled_df, X_test_scaled_df])
        y = pd.concat([y_train, y_validation, y_test])

        entry = (X, y)
        preprocessed_NN_bin.append(entry)
    preprocessed_NN_data.append(preprocessed_NN_bin)

# Save the data
with open(f'preprocessed_data/preprocessed_LSTM_CNN.pkl', 'wb') as f:
    pickle.dump(preprocessed_NN_data, f)

In [64]:
with open('processed_data/preprocessed_LSTM_CNN.pkl', 'rb') as f:
    LSTM_CNN_preprocessed_data = pickle.load(f)

In [65]:
print(X_train.shape, X_validation.shape, X_test.shape)

(981, 61) (123, 61) (124, 61)
